In [2]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L).

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [3]:
import os

directory = os.getcwd()

tr_mcc_codes = pd.read_csv(filepath_or_buffer=f"{directory}\\data\\tr_mcc_codes.csv", sep=";")
tr_types = pd.read_csv(filepath_or_buffer=f"{directory}\\data\\tr_types.csv", sep=";")
transactions = pd.read_csv(filepath_or_buffer=f"{directory}\\data\\transactions.csv", sep=",", nrows=1000000)
gender_train = pd.read_csv(filepath_or_buffer=f"{directory}\\data\\gender_train.csv", sep=",")

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [42]:
sample = transactions.sample(n=1000)

pos_atm_types = tr_types[(tr_types["tr_description"].str.contains("POS")) | (tr_types["tr_description"].str.contains("АТМ"))].tr_type
pos_atm_transactions = sample[sample.tr_type.isin(pos_atm_types)]

print(len(pos_atm_transactions)/len(sample))
pos_atm_types

0.622


5      4010
6      4011
7      4020
8      4021
9      4030
       ... 
126    2352
132    2401
136    2411
140    2421
145    2431
Name: tr_type, Length: 62, dtype: int64

## Задание 2


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [18]:
count = transactions.groupby(["tr_type"])
frequency = (count.size() / len(transactions)).sort_values(ascending=False).head(10)
print(frequency)

types = tr_types.set_index("tr_type").reindex(frequency.index).reset_index()
types

tr_type
1010    0.231117
2010    0.151166
7070    0.149006
1110    0.137658
1030    0.118975
2370    0.049830
7010    0.028972
7030    0.026078
7071    0.015781
1100    0.015476
dtype: float64


,tr_type,tr_description
0,1010,Покупка. POS ТУ СБ РФ
1,2010,Выдача наличных в АТМ Сбербанк России
2,7070,Перевод на карту (с карты) через Мобильный бан...
3,1110,Покупка. POS ТУ Россия
4,1030,Оплата услуги. Банкоматы СБ РФ
5,2370,Списание с карты на карту по операции <перевод...
6,7010,Взнос наличных через АТМ (в своем тер.банке)
7,7030,Перевод на карту (с карты) через АТМ (в предел...
8,7071,Перевод на карту (с карты) через Мобильный бан...
9,1100,Покупка. ТУ Россия


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [43]:
incomes = transactions[transactions.amount > 0].groupby(["customer_id"]).amount.sum().sort_values(ascending=False)

expenses = transactions[transactions.amount < 0].groupby(["customer_id"]).amount.sum().sort_values()
print(f"most deposited: {incomes.values[0]} by {incomes.index[0]}\nmost spent: {abs(expenses.values[0])} by {expenses.index[0]}")

print(f"модуль разницы: {abs(incomes.values[0] + expenses.values[0])}")

most deposited: 1248114886.81 by 70780820
most spent: 1249952204.79 by 70780820
модуль разницы: 1837317.980000019


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [56]:
print(frequency)
part1 = transactions[transactions["tr_type"].isin(frequency.index)].groupby("tr_type")
print("Среднее значение")
part1['amount'].mean()


tr_type
1010    0.231117
2010    0.151166
7070    0.149006
1110    0.137658
1030    0.118975
2370    0.049830
7010    0.028972
7030    0.026078
7071    0.015781
1100    0.015476
dtype: float64
Среднее значение


tr_type
1010    -19784.748640
1030     -5320.980222
1100    -44061.827262
1110    -32119.330371
2010   -136077.629325
2370   -205418.249032
7010    276391.789596
7030     86104.332909
7070     65569.831700
7071     66806.826623
Name: amount, dtype: float64

In [55]:
print("Медиана")
part1['amount'].median()

Медиана


tr_type
1010     -7411.52
1030     -2245.92
1100    -10188.26
1110    -11207.57
2010    -44918.32
2370    -44918.32
7010    112295.79
7030     13951.52
7070     11319.42
7071      3593.47
Name: amount, dtype: float64

In [58]:
part2 = transactions[transactions["customer_id"].isin([incomes.index[0], expenses.index[0]])].groupby("tr_type")
print("Среднее значение")
part2["amount"].mean()

Среднее значение


tr_type
2010   -2.941056e+06
2330   -2.382398e+06
2370   -2.218505e+06
6110    1.756293e+04
7020    1.465129e+04
7021    2.850067e+04
7030    1.406196e+04
7031    1.921036e+04
7034    1.430648e+04
7040    2.153833e+04
7070    1.384821e+04
7071    1.860057e+04
7074    2.281159e+04
Name: amount, dtype: float64

In [59]:
print("Медиана")
part2["amount"].median()

Медиана


tr_type
2010   -3368873.660
2330   -2245915.770
2370   -2245915.770
6110      10028.010
7020      10319.985
7021      28500.670
7030       8848.910
7031      12992.620
7034      14306.480
7040      14396.320
7070       8714.150
7071      12543.440
7074       9387.930
Name: amount, dtype: float64

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [62]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.head(1)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том,
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [263]:
expense_0 = transactions[(transactions['gender'] == 0) & (transactions['amount'] < 0)].amount
expense_1 = transactions[(transactions['gender'] == 1) & (transactions['amount'] < 0)].amount
print(f"Разница между средними тратами: {abs(expense_0.mean() - expense_1.mean())}")

income_0 = transactions[(transactions['gender'] == 0) & (transactions['amount'] > 0)].amount
income_1 = transactions[(transactions['gender'] == 1) & (transactions['amount'] > 0)].amount
print(f"Разница между средними поступлениями: {abs(income_0.mean() - income_1.mean())}")

Разница между средними тратами: 32718.054920224655
Разница между средними поступлениями: 63366.57104801461


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income".
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин.
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов.

In [86]:
tr_hour = pd.DataFrame({
    "tr_hour": transactions["tr_datetime"],
    "amount": transactions["amount"],
    "gender": transactions["gender"],
    "mcc_code": transactions["mcc_code"],
    "mcc_description": transactions["mcc_description"]
})

tr_hour["tr_hour"] = tr_hour["tr_hour"].str.slice(start=-8, stop=-6).astype(int)

night_0 = tr_hour.query("amount < 0 & gender == 0 & tr_hour >= 0 & tr_hour <= 6")
night_1 = tr_hour.query("amount < 0 & gender == 1 & tr_hour >= 0 & tr_hour <= 6")
print(f"Для пола 1: {len(night_0)}")
print(f"Для пола 2: {len(night_1)}")

Для пола 1: 43510
Для пола 2: 46694


## Доп. задание

найдите 5 категорий транзакций, по которым совершается больше всего трат в ночное время (>00:00 и < 6:00). Выведите названия категорий и максимальную сумму трат в каждой.

In [87]:
df = tr_hour.query("tr_hour >= 0 & tr_hour <= 6 & amount < 0")
categories = df.groupby("mcc_code")["amount"].sum().sort_values().head(5)
to_print = df[df["mcc_code"].isin(categories.index)].groupby("mcc_code")[["amount", "mcc_description"]]
to_print.min()

,amount,mcc_description
mcc_code,,
4829,-23748385.24,Денежные переводы
5411,-1190476.85,"Бакалейные магазины, супермаркеты"
5541,-474202.66,Станции техобслуживания
6011,-4491831.54,Финансовые институты — снятие наличности автом...
7011,-10376130.87,"Отели, мотели, базы отдыха, сервисы бронирования"


In [79]:
categories

tr_type
1110   -3.040334e+09
2010   -1.757425e+09
2370   -8.043574e+08
1100   -5.528784e+08
1210   -4.114363e+08
Name: amount, dtype: float64